# Gather data for figure 1,2,3

## Import packages

In [1]:
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as pn

import colicycle.time_mat_operations as tmo
from colicycle.MoMAobj import Momaobj
import colicycle.tools_GW as tgw

import scipy.optimize
import scipy.stats

## Define experimental parameters

In [2]:
#reference folder containing the folder PreProcessed
folder_prefix = '../'

#define a dictionary with conversion factors
expinfo = {}
expinfo['time_scale'] = 3#in min
expinfo['size_scale'] = 0.065

#define a list of folders containinig data
data_folders = [folder_prefix+'PreProcessed/20180706_GW296_glycerol37_1_MMStack/',
                folder_prefix+'PreProcessed/20180711_GW296_glucose37_1_MMStack/',
                folder_prefix+'PreProcessed/20180709_GW296_glucose8aa37_1_MMStack/',
                ]

## Load data, remove bad growth curves and complete cell cycle parameters

In [5]:
#load data and transform lengths in microns
colidata = [pd.read_pickle(x+'colidata.pkl') for x in data_folders]
for i in range(0,len(data_folders)):
    colidata[i]['mLb_fit'] = colidata[i].apply(lambda row: tmo.mother_var(row, colidata[i], 'Lb_fit'), axis = 1)
    colidata[i] = colidata[i][colidata[i].pearson_log>0.95]
    colidata[i]['condition'] = i
    colidata[i][['DLi','Lb_fit','Ld_fit','Ld','Lb','Ld','Li','Li_fit','Li_old']] \
=colidata[i][['DLi','Lb_fit','Ld_fit','Ld','Lb','Ld','Li','Li_fit','Li_old']].applymap(lambda x: x*expinfo['size_scale'])
    colidata[i]['DeltaL'] = colidata[i]['Ld_fit']-colidata[i]['Lb_fit']
    colidata[i]['DLdLi'] = colidata[i]['Ld_fit']-colidata[i]['Li_fit']
    colidata[i]['Tdi'] = colidata[i]['Td']-colidata[i]['Ti']
    
#combine datasets and remove bad fits
colidata_plot = pd.concat(colidata)
expinfo_pd = pd.Series(expinfo)

## Keep only variables relevant for plotting

In [7]:
colidata_export = colidata_plot[['Lb_fit','DeltaL','Li_fit','Li_old','DLi','tau_fit',
                                 'Tdi','DLdLi','mLb_fit','condition']].copy()


## Change condition name from numerics to text

In [8]:
def change_name(x):
    if x == 0:
        newname = 'glycerol'
    elif x==1:
        newname = 'glucose'
    elif x==2:
        newname = 'glucose8a'
    return newname

In [9]:
colidata_export['condition'] = colidata_export.condition.apply(lambda x: change_name(x))

## Export data

In [10]:
colidata_export = colidata_export.rename(columns = {'Lb_fit': 'Lb', 'DeltaL':'dL','Li_fit':'Li','Li_old':'Li_prev',
                        'DLi':'dLambda_if','tau_fit':'lambda_inv','Tdi':'Tid','DLdLi':'dLambda_ib','mLb_fit':'Lb_mother'
                        })
colidata_export.to_csv(folder_prefix+'Data_export/Fig1_2_3.csv', index = False)

